In [ ]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

In [ ]:
dirs = os.listdir('_data/raw/juros-bancarios')
dirs.sort()

all_files = []

for dir in dirs:
	for root, _, files in os.walk(f'_data/raw/juros-bancarios/{dir}'):
		for file in files:
			all_files.append(os.path.join(root, file))

arrow_tables = []
for file in all_files:
	with open(file, 'r') as f:
		print(f'Lendo arquivo {file}')
		data = eval(f.read())
		df = pd.DataFrame(data['conteudo'])
		table = pa.Table.from_pandas(df)
		arrow_tables.append(table)

combined_table = pa.concat_tables(arrow_tables)

df = combined_table.to_pandas()

In [3]:
df.drop('Posicao', axis=1, inplace=True)
df['TaxaJurosAoMes'] = df['TaxaJurosAoMes'].str.replace(',', '.').astype(float)
df['TaxaJurosAoAno'] = df['TaxaJurosAoAno'].str.replace(',', '.').astype(float)

In [ ]:
df['InstituicaoFinanceira'].value_counts()

In [ ]:
df.head()

In [ ]:
df['Modalidade'].value_counts()

In [7]:
df['SubClassificacao'] = df['Modalidade'].apply(lambda x: x.split(' - ')[-1])
df['Modalidade'] = df['Modalidade'].apply(lambda x: x.rsplit(' - ', 1)[0])

In [ ]:
df['Modalidade'].value_counts()

In [ ]:
df['SubClassificacao'].value_counts()

In [10]:
def transform_taxa(valor_taxa, periodo_desejado, periodo_atual):
	vt = Decimal(valor_taxa)
	pd = Decimal(periodo_desejado)
	pa = Decimal(periodo_atual)
 
	a = 1 + (vt / 100)
	b = pd / pa
	c = a ** b
	d = (c - 1) * 100

	return d

In [11]:
def trim_decimal(n):
	valor = Decimal(str(n).rstrip('0').rstrip('.'))
	return valor.quantize(Decimal('1.0000'), rounding=ROUND_HALF_UP)

In [12]:
df['TaxaJurosAoMes'] = df.apply(lambda x: trim_decimal(transform_taxa(x['TaxaJurosAoAno'], 1, 12)), axis=1)